# What is about 

Study degree distribution of KNN graphs for model data clouds - Gauss, uniform, sphere.
For high dimensions it seems to be  power law like. We also see drastical difference between sphere-torus vs Gauss-Uniform.
See discussion in : 
https://cstheory.stackexchange.com/questions/47957/power-law-for-degree-distribution-of-random-knn-graphs
see also https://stats.stackexchange.com/questions/500250/recognize-distribution-on-integers-0-mean-5-median-2-percentiles-75-5

V20 - same as 18,19 - but simulate Gauss, Uniform, Sphere, Torus 

V19 - same as 18, but trials increase to 100, and dim 1000 repeated twice with n_sample 1e4,1e5,

V18 - high dims explore: dim 100,1000,10000, K =5 n_sample = 1e4, Gauss 

V17 uniform , dim = 50, K = 10 

V16 same with , dim = 10, K = 20  strange thing  - mean sometimes not K for big sample size - is not a bug in cuml ? 
for dim 10 we see mean equals to median , while for dim 50 we saw mean was = percentile 75 

V15 same with , dim = 10, K = 10

V14 same with , dim = 10, K = 5

V13: K = 20

V12: repeat V11 for K = 10

V11 - study distribution in details for particular case - K=5, dim = 50 , change n_samples, look at percentiles, power law etc..  Findings summarized here: https://stats.stackexchange.com/questions/500250/recognize-distribution-on-integers-0-mean-5-median-2-percentiles-75-5

V10 - same as V9 - increase samples to 10_000_000 - crashed by memory after processing first case 

V9 - same as V8 - increase samples to 1_000_000 - 150 seconds 

V8 - simulations for Sphere and Torus added. See drastical difference for sphere-torus VS Gauss-uniform. 100_000 samples - 7 seconds 


V7 - same as V6 , but n_sample = 1e6 - run out of time 9 hours


V6 - linear interplotation of loglog plot - estimate exponent of power law of the distibution 

V5 - same as V4 but compare GPU and sklearn implementations - should coincide - indeed coincide 


V4 - pdf of degrees K=5, various dimensions , n_sample = 1e4

V3: plot pdf for :

Gauss dim50 n_neighbors3 n_sample100000 2276.7 2276.7 seconds passed trial, total

Gauss dim50 n_neighbors10 n_sample100000 2294.3 4571.1 seconds passed trial, total

V2: plot pdf for :

Gauss dim10 n_neighbors5 n_sample100000 73.2 73.2 seconds passed trial, total

Gauss dim50 n_neighbors5 n_sample100000 2313.5 2386.7 seconds passed trial, total

V1 - fast calculation:

Gauss dim10 n_neighbors5 n_sample10000 1.5 1.5 seconds passed trial, total

Gauss dim50 n_neighbors5 n_sample10000 11.9 13.4 seconds passed trial, total


dim10 n_neighbors5 n_sample1000 0.1 0.1 seconds passed trial, total

dim10 n_neighbors5 n_sample10000 1.5 1.6 seconds passed trial, total

dim10 n_neighbors5 n_sample100000 66.1 67.7 seconds passed trial, total

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objs as go

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.neighbors import kneighbors_graph
import time
import matplotlib.pyplot as plt
import plotly
_t00 = time.time()



In [ ]:
import numpy as np
from scipy.stats import rv_continuous
import scipy.integrate as integrate
import scipy.special as special


class CustomDistribution(rv_continuous):
    def __init__(self, momtype = 'pdf', a=0, b=1):
        super().__init__(a=a, b=b)
    
    def _get_support(*args, **kwargs):
        return 0, 1
    
    def _pdf(self, x):
        if x < 0 or x > 1:
             return 0
            
        integral = integrate.quad(lambda t: np.sinh(t), 0, 1)[0]
        res = (np.sinh(x)) / integral
        return res


n_sample = 10000
t0 = CustomDistribution().rvs(size=n_sample)
t1 = np.random.rand(n_sample) * 2 * np.pi
X0 = np.cosh(t0)
X1 = np.sinh(t0) * np.cos(t1)
X2 = np.sinh(t0) * np.sin(t1)


In [ ]:

x = X0
y = X1
z = X2
trace = go.Scatter3d(
   x = x, y = y, z = z,mode = 'markers', marker = dict(
      size = 6,
      color = z, # set color to an array/list of desired values
      colorscale = 'Viridis'
      )
   )
layout = go.Layout(title = '3D Scatter plot')
fig = go.Figure(data = [trace], layout = layout)
fig.show()

In [ ]:
print(f"V_1/V_2 = {np.sinh(1/4) ** 2 / np.sinh(1/2) ** 2}")
print(f"N_1/N_2 = {X0[X0 < np.cosh(1/2)].size / X0.size}")

In [ ]:
try:
    import cudf
    from cuml.neighbors import NearestNeighbors
    #from cuml.datasets import make_blobs    
except:
    print('GPU library cannot be imported. Turn ON GPU')

In [ ]:

#distribution_type = 'Gauss' # 'uniform'
verbose = 0


plt.style.use('ggplot')

fig = plt.figure(figsize=(20,12))

#ax2 =     fig.add_subplot(2,1,2)

t00=time.time()

df_stat = pd.DataFrame()
df_stat2 = pd.DataFrame()

list_save_results = []
n_sample0 = 1e4
i = 0
#for dim , n_neighbors, n_sample  in [(2,5,1e4), (5,5,1e4),   (8,5,1e4),   (10,5,1e4), (50,5,1e4), (500,5,1e4)]: #  , (100,10,1e4)  , (10,5,1e4) , (10,5,1e5) ]:  
for distribution_type in ['Gauss', 'Uniform', 'Sphere', 'Torus' ]: #  ['Gauss']: # 
    i += 1
    #ax1 =     
    #fig.add_subplot(2,1,i)
    fig = plt.figure(figsize=(20,8))
    configs = [(x, 5, n_sample0) for x in [100, 1000, 10000]]
    
#     for dim , n_neighbors, n_sample  in [(100,5,n_sample0), (1000,5,n_sample0), (1000,5,10*n_sample0),  (10000,5,n_sample0),  ]: # , (50,10,10*n_sample0) , (50,10,100*n_sample0) ]: # ,(500,5,n_sample0),]:# [(2,5,n_sample0),     (10,5,n_sample0), (50,5,n_sample0),(500,5,n_sample0)]: 
#                                           #  , (5,5,n_sample0) , (100,10,1e4)  , (10,5,1e4) , (10,5,1e5) ]:  
    
    for dim , n_neighbors, n_sample  in configs:
        n_sample = int(n_sample)
        df_stat_over_trials = pd.DataFrame()
        
        for trial in range(100):    
            if distribution_type == 'Gauss':
                X = np.random.randn(n_sample,dim)
                if verbose >= 100:
                    print('Gauss')
            elif distribution_type == 'Sphere':
                X = np.random.randn(n_sample,dim+1)
                s = np.sqrt( (X*X).sum(axis=1))
                X = X / s[:,np.newaxis]
                if verbose >= 100:
                    print('Sphere')
            elif distribution_type == 'Torus':
                X = ( np.random.rand(n_sample,dim) )
                X1 = np.sin(2*np.pi*X)
                X2 = np.cos(2*np.pi*X)
                X = np.concatenate((X1,X2),axis=1)
                if verbose >= 100:
                    print('Torus')
            elif distribution_type == 'Parabolic':
                t0 = np.random.rand(n_sample)
                t1 = np.random.rand(n_sample) * 2 * np.pi
                X0 = np.cosh(t0)
                X1 = np.sinh(t0) * np.cos(t1)
                X2 = np.sinh(t0) * np.sin(t1)
                if verbose >= 100:
                    print('Parabolic')
            else:
                X = np.random.uniform(-1,1, size = (n_sample,dim) )
                if verbose >= 100:
                    print('Uniform')    

            for method in ['GPU']: # 'sklearn',  'GPU'

                t0=time.time()

                if method in ['GPU']:
                    X_cudf = cudf.DataFrame(X)
                    model = NearestNeighbors(n_neighbors=n_neighbors+1)
                    model.fit(X)
                    # get 3 nearest neighbors
                    m = model.kneighbors_graph(X_cudf)#, include_self=False) #  no include_self
                    vec_degs = np.sum(m,axis = 0 )
                    vec_degs = vec_degs.get()

                    vec_degs = np.squeeze(np.array(vec_degs.ravel()))
                    vec_degs -= np.ones_like(vec_degs ) # no include_self - so that is a way round

                else: #if method in ['sklearn']:
                    m = kneighbors_graph(X, n_neighbors=n_neighbors, mode='connectivity' , include_self=False) #'distance' mode=  'connectivity'
                    vec_degs = np.sum(m,axis = 0 )
                    vec_degs = np.squeeze(np.array(vec_degs.ravel()))

                #print(m.toarray())

                bins = np.arange(200)# np.max(vec_degs))
                h = np.histogram(vec_degs,bins = bins, density= True)

                #label = method +' '+ distribution_type + ' dim'+str(dim) +' n_neighbors'+str(n_neighbors) + ' n_sample'+str(n_sample)
                label = distribution_type + ' dim'+str(dim) +' n_neighbors'+str(n_neighbors) + ' n_sample'+str(n_sample)

                list_save_results.append( (label, h) )

                #plt.plot(h[0],'*-',label = label)
                #plt.loglog(h[0],'*-',label = label)
                #plt.legend(fontsize = 13)
                #plt.xlabel('Degree')
                #plt.ylabel('Probability')

                degs = bins[:-1]
                probs = h[0]
                # Cut suitable subset to make linear interpolation (i.e. avoid noisy part) and take logs:   
                m = (degs > 2*n_neighbors ) # & (degs < 90)
                y = (probs[m])
                x = (degs[m])
                for t in range(len(y) ):
                    if y[t] == 0:
                        break
                t = t-1
                y = y[:t]
                x = x[:t]
                y = np.log10(y)
                x = np.log10(x)

                coefs_polyfit = np.polyfit(x, y, 1)
                x2 =  np.log10(degs[degs > 0])
                vec_line_intepolation_result = np.poly1d(coefs_polyfit)(x2)
                if verbose >= 100:
                    print(label, 'coefficients of line interpolation: ', coefs_polyfit )
                m = h[0] > 1e-12
                if trial <= 1:
                    #plt.loglog(h[1][:-1][m], h[0][m],'*-', label = label, linewidth=4)
                    plt.loglog( h[0][m],'*-', label = label, linewidth=4)
                    plt.loglog(np.power(10,x2), np.power(10, vec_line_intepolation_result ), label = 'linear approx' )#  ,  label = label+' Approx', linewidth=4)
                    plt.title('LogLog plot')
                    plt.xlabel('degree')
                    #plt.ylabel('Node count')
                    plt.legend( fontsize = 12 )
                    #plt.show()

                    #ax1.grid()

                    #ax2.loglog(h[0],'*-',label = label)
                    #ax2.legend()
                    #ax2.grid()

                st = pd.Series(vec_degs).describe( percentiles = [0.05,0.1, .25, .5, .75, 0.9,0.95,0.99 ])
                st.name = label + ' trial' + str(trial)
                st['Linear slope'] = coefs_polyfit[0]
                st['Linear constant'] = coefs_polyfit[1]
                st['Seconds passed'] = np.round( time.time()-t0, 2)
                df_stat = df_stat.join( st, how = 'outer'  )
                df_stat_over_trials = df_stat_over_trials.join( st, how = 'outer'  )
                
                if verbose >= 100:
                    print(label,  np.round(time.time()-t0,1),  np.round(time.time()-t00,1),'seconds passed trial, total')
                
        label = distribution_type + ' dim'+str(dim) +' n_neighbors'+str(n_neighbors) + ' n_sample'+str(n_sample)
        st = df_stat_over_trials.mean(axis = 1)
        st.name = label 
        df_stat2 = df_stat2.join( st, how = 'outer'  ) 
        st = df_stat_over_trials.std(axis = 1)
        st.name = label + ' std' 
        df_stat2 = df_stat2.join( st, how = 'outer'  ) 

plt.show()
    
seconds_passed_total = time.time()-t00
print( np.round(seconds_passed_total,1),  np.round(seconds_passed_total/60,1), np.round(seconds_passed_total/3600,1), 
      'seconds, minutes, hours passed')

#print(df_stat)
df_stat2

In [ ]:
print( np.round(seconds_passed_total,1),  np.round(seconds_passed_total/60,1), np.round(seconds_passed_total/3600,1), 
      'seconds, minutes, hours passed')


In [ ]:
df_stat2

In [ ]:
df_stat

In [ ]:
df_stat2.to_csv('df_stat2.csv')
df_stat.to_csv('df_stat.csv')


In [ ]:
if 0:
    fig = plt.figure(figsize=(20,12))

    for label,h in list_save_results:
        plt.plot(h[0][h[0]>0],'*-', label = label)

    plt.legend()
    #plt.grid()
    plt.show()

In [ ]:
if 0:
    fig = plt.figure(figsize=(20,12))

    for label,h in list_save_results:
        plt.loglog(h[0][h[0]>1e-12],'*-', label = label)

    plt.legend()
    #plt.grid()
    plt.show()

In [ ]:
print(np.round(time.time()- _t00,1), np.round( (time.time()-_t00)/60,1),  np.round( (time.time()-_t00)/3600,1), 'seconds minutes hours total passed')
